In [20]:
import utils
import plotting
import postprocessing
import corrections

from utils import CUT_MAX_VAL
from hh_vars import (
    years,
    sig_keys,
    data_key,
    qcd_key,
    bg_keys,
    samples,
    res_samples,
    res_sig_keys,
    txbb_wps,
    jec_shifts,
    jmsr_shifts,
)
from postprocessing import (
    shape_var,
    shape_bins,
    blind_window,
    selection_regions,
    selection_regions_label,
    # selection_regions_year,
)

import numpy as np
import pandas as pd
import pickle
from pandas.errors import SettingWithCopyWarning
from hist import Hist

import os
from copy import deepcopy
from inspect import cleandoc
import warnings

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
sig_split_points = [
    [
        (1000, 125),
        (1400, 125),
        (1800, 125),
        (2200, 125),
        (3000, 125),
    ],
    [
        (1400, 150),
        (1800, 150),
        (1800, 190),
        (2200, 150),
        (2200, 190),
        (2200, 250),
    ],
    [
        (3000, 125),
        (3000, 150),
        (3000, 190),
        (3000, 250),
        (3000, 350),
    ],
]

sig_splits = [
    ["HHbbVV"] + [f"X[{mX}]->H(bb)Y[{mY}](VV)" for (mX, mY) in mps] for mps in sig_split_points
]

In [11]:
MAIN_DIR = "../../../"
samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb24"
signal_samples_dir = f"{MAIN_DIR}/../data/skimmer/Mar10_2"
year = "2017"

plot_dir = f"../../../plots/PostProcessing/Mar11/"
templates_dir = "templates/Mar11/"
for i in range(len(sig_splits)):
    _ = os.system(f"mkdir -p {plot_dir}/ControlPlots/{year}/sigs{i}/")
_ = os.system(f"mkdir -p {plot_dir}/cutflows/")
_ = os.system(f"mkdir -p {plot_dir}/templates/")
# _ = os.system(f"mkdir -p {templates_dir}")

Load samples

In [6]:
# Both Jet's Regressed Mass above 50, electron veto
filters = [
    [
        ("('ak8FatJetParticleNetMass', '0')", ">=", 50),
        ("('ak8FatJetParticleNetMass', '1')", ">=", 50),
        # ("('nGoodElectrons', '0')", "==", 0),
    ],
]
systematics = {}

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()) + list(res_samples.keys()))

# utils.remove_empty_parquets(samples_dir, year)
events_dict = utils.load_samples(signal_samples_dir, res_samples, year, filters)
events_dict |= utils.load_samples(samples_dir, samples, year, filters)

utils.add_to_cutflow(events_dict, "BDTPreselection", "weight", cutflow)

print("")
# print weighted sample yields
for sample in events_dict:
    tot_weight = np.sum(events_dict[sample]["weight"].values)
    print(f"Pre-selection {sample} yield: {tot_weight:.2f}")

Loaded NMSSM_XToYH_MX1000_MY125_HTo2bYTo2W_hadronicDecay : 39510 entries
Loaded NMSSM_XToYH_MX1400_MY125_HTo2bYTo2W_hadronicDecay : 55362 entries
Loaded NMSSM_XToYH_MX1400_MY150_HTo2bYTo2W_hadronicDecay : 55866 entries
Loaded NMSSM_XToYH_MX1800_MY125_HTo2bYTo2W_hadronicDecay : 61954 entries
Loaded NMSSM_XToYH_MX1800_MY150_HTo2bYTo2W_hadronicDecay : 63425 entries
Loaded NMSSM_XToYH_MX1800_MY190_HTo2bYTo2W_hadronicDecay : 61065 entries
Loaded NMSSM_XToYH_MX2200_MY125_HTo2bYTo2W_hadronicDecay : 64229 entries
Loaded NMSSM_XToYH_MX2200_MY150_HTo2bYTo2W_hadronicDecay : 63686 entries
Loaded NMSSM_XToYH_MX2200_MY190_HTo2bYTo2W_hadronicDecay : 62670 entries
Loaded NMSSM_XToYH_MX2200_MY250_HTo2bYTo2W_hadronicDecay : 58108 entries
Loaded NMSSM_XToYH_MX3000_MY125_HTo2bYTo2W_hadronicDecay : 66929 entries
Loaded NMSSM_XToYH_MX3000_MY150_HTo2bYTo2W_hadronicDecay : 71414 entries
Loaded NMSSM_XToYH_MX3000_MY190_HTo2bYTo2W_hadronicDecay : 69635 entries
Loaded NMSSM_XToYH_MX3000_MY250_HTo2bYTo2W_hadronic

Scale factors and bb VV assignment

In [7]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)
cutflow


QCD_SCALE_FACTOR = 0.8144326761300358


,BDTPreselection,TriggerEffs,QCD SF
HHbbVV,3.087472e+00,1.781177e+00,1.781177e+00
QCD,2.300678e+06,1.196074e+06,9.741217e+05
TT,1.565833e+05,1.012822e+05,1.012822e+05
ST,1.094484e+04,7.371344e+03,7.371344e+03
V+Jets,6.356934e+04,3.891328e+04,3.891328e+04
Diboson,9.876074e+02,6.254659e+02,6.254659e+02
Data,1.122314e+06,1.122314e+06,1.122314e+06
X[1000]->H(bb)Y[125](VV),1.953725e+01,1.650731e+01,1.650731e+01
X[1400]->H(bb)Y[125](VV),2.740380e+01,2.648171e+01,2.648171e+01
X[1400]->H(bb)Y[150](VV),2.645352e+01,2.561982e+01,2.561982e+01


Control Plots

In [ ]:
# {var: (bins, label)}
control_plot_vars = {
    "MET_pt": ([50, 0, 300], r"$p^{miss}_T$ (GeV)"),
    "DijetEta": ([50, -8, 8], r"$\eta^{jj}$"),
    "DijetPt": ([50, 0, 750], r"$p_T^{jj}$ (GeV)"),
    "DijetMass": ([50, 500, 3200], r"$m^{jj}$ (GeV)"),
    "bbFatJetEta": ([50, -2.4, 2.4], r"$\eta^{bb}$"),
    "bbFatJetPt": ([50, 300, 1500], r"$p^{bb}_T$ (GeV)"),
    "bbFatJetParticleNetMass": ([50, 0, 300], r"$m^{bb}_{reg}$ (GeV)"),
    "bbFatJetMsd": ([50, 0, 300], r"$m^{bb}_{msd}$ (GeV)"),
    "bbFatJetParticleNetMD_Txbb": ([50, 0.8, 1], r"$p^{bb}_{Txbb}$"),
    "VVFatJetEta": ([50, -2.4, 2.4], r"$\eta^{VV}$"),
    "VVFatJetPt": ([50, 300, 1500], r"$p^{VV}_T$ (GeV)"),
    "VVFatJetParticleNetMass": ([50, 0, 300], r"$m^{VV}_{reg}$ (GeV)"),
    "VVFatJetMsd": ([50, 0, 300], r"$m^{VV}_{msd}$ (GeV)"),
    "VVFatJetParticleNet_Th4q": ([50, 0, 1], r"Prob($H \to 4q$) vs Prob(QCD) (Non-MD)"),
    "VVFatJetParTMD_THWW4q": (
        [50, 0, 1],
        r"Prob($H \to VV \to 4q$) vs Prob(QCD) (Mass-Decorrelated)",
    ),
    "VVFatJetParTMD_probT": ([50, 0, 1], r"Prob(Top) (Mass-Decorrelated)"),
    "bbFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{bb}_T / p_T^{jj}$"),
    "VVFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{VV}_T / p_T^{jj}$"),
    "VVFatJetPtOverbbFatJetPt": ([50, 0.4, 2.0], r"$p^{VV}_T / p^{bb}_T$"),
    "nGoodMuons": ([3, 0, 3], r"# of Muons"),
    "nGoodElectrons": ([3, 0, 3], r"# of Electrons"),
    "nGoodJets": ([5, 0, 5], r"# of AK4 B-Jets"),
}

hists = postprocessing.control_plots(
    events_dict,
    bb_masks,
    sig_keys + res_sig_keys,
    control_plot_vars,
    f"{plot_dir}/ControlPlots/{year}/",
    year,
    sig_splits=sig_splits,
    # show=True,
)

Overall BDT SF

In [ ]:
sel, cf = utils.make_selection(
    selection_regions[year]["lpsf"], events_dict, bb_masks, prev_cutflow=cutflow
)
lp_sf, unc, uncs = postprocessing.get_lpsf(events_dict[sig_key], sel[sig_key])
print(f"BDT Scale Factor: {lp_sf:.2f} ± {unc:.2f}")
print(uncs)
systematics["lp_sf_unc"] = unc / lp_sf
cf

Templates

In [ ]:
postprocessing.get_templates(
    events_dict,
    bb_masks,
    year,
    selection_regions[year],
    shape_var,
    shape_bins,
    blind_window,
    plot_dir=plot_dir,
    prev_cutflow=cutflow,
    weight_shifts={},
    jshift="",
    show=True,
)

In [ ]:
templates = {}

for jshift in [""] + jec_shifts + jmsr_shifts:
    print(jshift)
    ttemps, tsyst = postprocessing.get_templates(
        events_dict,
        bb_masks,
        year,
        selection_regions[year],
        shape_var,
        shape_bins,
        blind_window,
        plot_dir=plot_dir,
        prev_cutflow=cutflow,
        weight_shifts=postprocessing.weight_shifts,
        jshift=jshift,
        show=False,
    )

    templates = {**templates, **ttemps}
    systematics = {**systematics, **tsyst}

In [ ]:
systematics

In [ ]:
templates_dict = {}

for year in years:
    with open(f"templates/Feb28/{year}_templates.pkl", "rb") as f:
        templates_dict[year] = pickle.load(f)